In [1]:
import rasterio
import xarray as xr


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from  matplotlib.cm import ScalarMappable
from  matplotlib.colors import ListedColormap, BoundaryNorm

import cartopy.crs as ccrs
import cartopy.feature as cfeature
%matplotlib inline

C:\Users\rebek\anaconda3\envs\envi5809\Lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [3]:
import rioxarray as rio
# from rasterio.plot import show
# from rasterio.windows import from_bounds

ModuleNotFoundError: No module named 'rasterio.mask'

In [7]:
install rasterio

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement rasterio.mask (from versions: none)
ERROR: No matching distribution found for rasterio.mask


In [ ]:
def vegetation_area_analysis(start_year,end_year,sample_site,site_x_origin,site_y_origin,site_size):

    
    """
    
    This function creates an output of area calculations for each vegetation class for the sample site defined. 
    
    start_year = first year to calculate areas for
    end_year = final year to calculate areas for
    sample_site = sample site name for file naming
    site_x_origin = bottom left x-coordinate of sample site in geographic coordinates
    site_y_origin = bottom left y-coordinate of sample site in geographic coordinates
    site_size = size of sample site in geographic coordinates
    
    Datasources:
    
    1. The vegetation raster data is extracted from https://code.earthengine.google.com/?scriptPath=users%2Fmapbiomas%2Fuser-toolkit%3Amapbiomas-user-toolkit-lulc.js
    2. Legend codes for vegetation classes were taken from https://s3.amazonaws.com/amazonia.mapbiomas.org/leyenda/C%C3%B3digo_de_la_Leyenda_-_colecci%C3%B3n_4.pdf
    

    """
    
    year = start_year
    
    # Download the colour codes for vegetation classes - extract from a saved CSV file

    colorCode = pd.read_csv('Colour Codes for Vegetation.csv')
    colorCode.drop(columns=['ClassesBR'],inplace=True)
    colorCode
    
    # Extract original colours
    sortColor = colorCode.sort_values('ID')

    vals = sortColor['ID'].values
    cols = sortColor['HexCode'].values
    bounds = np.append(vals, vals[-1] + 1)

    cmap = ListedColormap(cols)
    norm = BoundaryNorm(bounds, ncolors=len(cols))

    
    while year <= end_year:

        #Loading in the dataset

        ds = rio.open_rasterio("https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_8/lclu/coverage/brasil_coverage_"+str(year)+".tif")


        # Slice to the sample site

        ds_clip = ds.sel(x=slice(site_x_origin,(site_x_origin + site_size)),y=slice((site_y_origin + site_size),site_y_origin)).isel(band=0)


        # Counting the area for the each vegetation class

        totalArea = 111**2 # in km2
        totalPix = ds_clip.size # number on pixel for the clipped dataset

        count = []
        for k in range(len(colorCode)):
            classID = colorCode['ID'].values[k]
            count.append(ds_clip.where(ds_clip==classID).count().values+0)

        colorCode['count'] = count
        colorCode['perc'] = np.round(np.asarray(count)*100/totalPix,3)
        colorCode[''+str(year)+''] = colorCode['perc'].values*totalArea/100.
        vegAnalysis = colorCode.copy()
        vegAnalysis.drop(columns=['HexCode'],inplace=True)
        vegAnalysis

        year += 1
        

    # Plot to csv
    vegAnalysis.to_csv('vegetation_analysis_sample_site_'+str(sample_site)+'_'+str(start_year)+'-'+str(end_year)+'.csv',index=False)

    return

In [ ]:
vegetation_area_analysis(2000,2022,'Control Site',-69,-3,2)

In [ ]:
vegetation_area_analysis(2000,2022,'Sample Site 1',-48,-5,2)

In [ ]:
vegetation_area_analysis(2000,2022,'Sample Site 2',-52,-4,2)

In [ ]:
vegetation_area_analysis(2000,2022,'Sample Site 3',-65,-10,2)